In [4]:

'''
Project: Neural Network for MHC Peptide Prediction
Class(s): BuildNetwork
Function: Generates specified neural network architecture (data agnostic)

Author: Patrick V. Holec
Date Created: 2/2/2017
Date Updated: 2/2/2017
'''


# standard libaries
import gzip
import math
import os.path
import time
import pickle
import random

# nonstandard libraries
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

# library modifications
random.seed(42)
tf.set_random_seed(42)

# main test function
def main():
    pass

# Factory methods for creating variables and layers

def weight_variable(shape):
    """Create a weight variable with appropriate initialization."""
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    """Create a bias variable with appropriate initialization."""
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W, stride=1, padding='SAME'):
    return tf.nn.conv2d(x, W, strides=[1, stride, stride, 1], padding=padding)

def max_pool(x, stride=2, filter_size=2):
    return tf.nn.max_pool(x, ksize=[1, filter_size, filter_size, 1],
                        strides=[1, stride, stride, 1], padding='SAME')

def cross_entropy(y, y_real):
    return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y, y_real))

def l2_loss(y,y_real):
    return 
    #return tf.nn.l2_loss(y - y_real)
    
def build_two_fc_layers(x_inp,Ws,bs):
    h_fc1 = tf.nn.relu(tf.matmul(x_inp, Ws[0]) + bs[0])
    return tf.matmul(h_fc1, Ws[1]) + bs[1]

'''
BuildNetwork: Main neural network architecture generator
'''

class BuildNetwork:
    def __init__(self,label=None,silent=False):
        print 'Initializing neural network data acquisition...'
        
        # check for label
        if not label:
            print 'No data label defined, unable to initialize any architecture.'
            return None
        
        # load data parameters
        data = LoadedData(label)
        self.__dict__.update(data.params)
        self.all_data,self.all_labels = data.data_array,data.label_array
        
        # set some basic hyperparameters
        self.test_fraction = 0.2
        self.batch_size = 10
        self.x_dim,self.y_dim = self.aa_count,self.length
        
        # update on model parameters
        if not silent:
            print '*** System Parameters'
            print '  - Sequence length:',self.length
            print '  - AA count:',self.aa_count
        
        print 'Finished acquisition!'
    
    def data_format(self,silent=False,**kwargs):
        print 'Starting data formatting...'
        
        # randomize data order
        order,limit = range(0,len(self.all_data)),int((1-self.test_fraction)*len(self.all_data))
        np.array(random.shuffle(order))
        
        # normalize label energy
        self.all_labels = np.reshape(np.array([(self.all_labels - min(self.all_labels))/
                                               (max(self.all_labels)-min(self.all_labels))]),
                                     (len(self.all_labels),1,1))
        
        # split data into training and testing
        self.train_data = self.all_data[np.array(order[:limit]),:,:]
        self.test_data = self.all_data[np.array(order[limit:]),:,:]
        self.train_labels = self.all_labels[np.array(order[:limit]),:,:]
        self.test_labels = self.all_labels[np.array(order[limit:]),:,:]
        
        self.train_data = np.reshape(self.train_data,(self.train_data.shape[0],self.x_dim*self.y_dim))
        self.test_data = np.reshape(self.test_data,(self.test_data.shape[0],self.x_dim*self.y_dim))
        self.train_labels = np.reshape(self.train_labels,(self.train_labels.shape[0],1))
        self.test_labels = np.reshape(self.test_labels,(self.test_labels.shape[0],1))
        
        print 'Finished formatting!'


    ''' 
    Provides output class variables:
    self.train_x,self.train_y
    '''
    
    def CNN_initialization(self,layers=2,learning_rate=0.01):
        print 'Constructing CNN graph...'
        
        # model hyperparameters
        conv1_filter_size = (self.aa_count,1)
        conv1_depth = self.aa_count # a filter for each position!
        conv2_filter_size = (3,3)
        conv2_depth = 5
        conv_stride = 1
        fc_num_hidden = self.aa_count*self.length
        
        # create model input structure
        self.train_x = tf.placeholder(tf.float32, shape=(None, self.x_dim*self.y_dim)) # vector input
        self.train_y = tf.placeholder(tf.float32, shape=(None, 1)) # one output (ddG)
        x_image = tf.reshape(self.train_x, [-1, self.x_dim, self.y_dim, 1])
        
        # weight initialization
        self.W1 = weight_variable([conv1_filter_size[0],conv1_filter_size[1],1,conv1_depth])
        self.W2 = weight_variable([conv2_filter_size[0],conv2_filter_size[1],conv1_depth,conv2_depth])
        

        
        # convolutional layer 1
        conv1 = conv2d(x_image, self.W1, stride=conv_stride,padding='VALID') # keeps dimensions at [x_dim,y_dim]
        #h_pool1 = max_pool(conv1, stride=2, filter_size=2) # keeps dimensions at [x_dim,y_dim]
        
        # convolutional layer 2
        conv2 = conv2d(conv1, self.W2, stride=conv_stride,padding='SAME')
        #h_pool2 = max_pool(conv2, stride=2, filter_size=2)
        
        # size of feature maps
        conv2_feat_map_x = int(conv2.get_shape()[2])   # Define the x-size of the conv2 feature map
        conv2_feat_map_y = int(conv2.get_shape()[1])   # Define the y-size of the conv2 feature map

        # weights/biases for fully connected layer 1
        self.W_fc1 = weight_variable([conv2_feat_map_x * conv2_feat_map_y * conv2_depth, fc_num_hidden])
        self.b_fc1 = bias_variable([fc_num_hidden])
        
        h_pool2_flat = tf.reshape(conv2, [-1, conv2_feat_map_x * conv2_feat_map_y * conv2_depth])
        
        # weights/biases for fully connected layer 2
        self.W_fc2 = weight_variable([fc_num_hidden, 1])
        self.b_fc2 = bias_variable([1])

        self.y_conv = build_two_fc_layers(h_pool2_flat, [self.W_fc1, self.W_fc2], [self.b_fc1, self.b_fc2])
        
        # model access variables
        #self.loss = cross_entropy(self.y_conv, self.train_y)
        self.loss = l2_loss(self.y_conv,self.train_y)
        self.train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(self.loss)
        
        print 'Finished construction!'
        
    
    # requires self.train_data,self.train_labels
    #          self.train_step,self.loss
    def train(self):
        # start timer
        start = time.time()
        
        # training hyperparameters
        num_epochs = 500
        batch_size = 50

        # training via iterative epochs
        batches_per_epoch = int(len(self.train_data)/batch_size)
        num_steps = int(num_epochs * batches_per_epoch)
        
        print 'Batchs per epoch - {} / Number of steps - {}'.format(batches_per_epoch,num_steps)
        
        sess = tf.Session()
        init = tf.global_variables_initializer()
        sess.run(init)
        print 'Initializing variables...'

        epoch_loss = 0
        epoch_acc = 0
        for step in xrange(num_steps):
            offset = (step * batch_size) % (self.train_data.shape[0] - batch_size)

            batch_x = self.train_data[offset:(offset + batch_size), :]
            batch_y = self.train_labels[offset:(offset + batch_size)]

            feed_dict = {self.train_x: batch_x, self.train_y: batch_y}
            
            _, batch_loss = sess.run([self.train_step, self.loss],feed_dict=feed_dict)
            
            epoch_loss += batch_loss
                        
            if (step % batches_per_epoch == 0):
                epoch_loss /= batches_per_epoch
                print 'Avg batch loss at step %d: %f' % (step, epoch_loss)
                epoch_loss = 0
                # randomize input data
                together = np.concatenate((self.train_data,self.train_labels),axis=1)
                np.random.shuffle(together)
                self.train_data = together[:,:-1]
                self.train_labels = np.reshape(together[:,-1],(self.train_labels.shape[0],1)) # need to add dimension to data
                
        print "Training time: ", time.time() - start
        
        sess.close()
        
    def visualization(self,picks=['test_accuracy','filters']):
        # start engine
        sess = tf.Session()
        init = tf.global_variables_initializer()
        sess.run(init)
        
        if 'test_accuracy' in picks:
            # visualize
            predicted_labels = sess.run(self.y_conv, feed_dict={self.train_x: self.test_data})
            plt.scatter(predicted_labels,self.test_labels)
            plt.show()
            
        if 'filters' in picks:
            # layer 1 weights
            A = sess.run(self.W1)
            print 'A:',A
            sh = A.shape
            A = np.reshape(A,(sh[0],sh[3]))
            if True:
                plt.imshow(A, cmap='jet', interpolation='nearest')
                plt.title('Filter (Layer 1)')
                plt.xlabel('AA Index')
                plt.ylabel('Filter #')
                plt.show()   
                
        sess.close()
        
        
class LoadedData:
    def __init__(self,label='test'):
        
        # open file and store lines
        with open('{}_seqs.txt'.format(label),'r') as f:
            content = f.readlines()
        
        # pull out parameters from the pickeled params file
        self.params = pickle.load(open('{}_params.p'.format(label),'rb'))
        
        # split up lines it data and labels
        self.raw_data,self.raw_labels = [],[]
        for data in [c.strip('\n').split(',') for c in content]:
            self.raw_data.append(data[0])
            self.raw_labels.append(float(data[1]))
            
        # one-hot encoding
        self.data_array = np.zeros((len(self.raw_data),self.params['aa_count'],self.params['length']),np.int)
        self.label_array = np.reshape(np.array(self.raw_labels),(len(self.raw_labels),1,1))
        # np.zeros((len(self.raw_labels),1,1),np.float32)
        for i,sample in enumerate(self.raw_data):
            for j,char in enumerate(sample):
                self.data_array[i,self.params['characters'].index(char),j] = 1
            
# namespace activation
if __name__ == '__main__':
    main()




In [5]:
network = BuildNetwork('test')
network.data_format()
network.CNN_initialization(learning_rate=0.00001)
network.train()
network.visualization()

Initializing neural network data acquisition...
*** System Parameters
  - Sequence length: 5
  - AA count: 6
Finished acquisition!
Starting data formatting...
Finished formatting!
Constructing CNN graph...
W1: (6, 1, 1, 6)
X_image: (?, 6, 5, 1)


AttributeError: 'NoneType' object has no attribute 'dtype'